# Amazon Sagemaker Reinforcement Learning demo

In [42]:
import os
import sys

import sagemaker
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

## Execute the training

In [63]:
# select in which AWS S3 bucket store the trained model
s3_bucket = 'fruitpunch-sagemaker-test'
s3_output_path = 's3://{}/'.format(s3_bucket)

In [54]:
# select whether to run the image locally
# (on a ml.t2.medium instance within the notebook) or
# on a dedicated larger instance
local = True

if local:
    train_type = 'local'
    role = 'arn:aws:iam::<ACCOUNT_ID>:user/<USERNAME>'
else:
    train_type = 'ml.m5.large'
    role = sagemaker.get_execution_role()

In [ ]:
# launch the training job
estimator = RLEstimator(entry_point='random_trainer.py',
                        source_dir='src',
                        image_name='<ACCOUNT_ID>.dkr.ecr.<REGION>.amazonaws.com/<REPOSITORY_NAME>:latest',
                        role=role,
                        train_instance_count=1,
                        train_instance_type=train_type,
                        base_job_name='rl_random_example',
                        output_path=s3_output_path)
estimator.fit()